In [3]:
from importlib import reload
import os 
import pathlib
import sys
sys.path.append('..')

from kerchunk.hdf import SingleHdf5ToZarr 
from kerchunk.combine import MultiZarrToZarr

import cartopy.crs as ccrs
import cartopy.feature as cfeature 
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import dask
from dask.distributed import Client
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
from mpl_toolkits.axes_grid1 import make_axes_locatable
import s3fs
import datetime as dt

import logging
import fsspec
import ujson
from tqdm import tqdm
from glob import glob
import seaborn as sb

from src import processing as pr, parallel_plotting as pp
reload(pr)
reload(pp)

<module 'src.parallel_plotting' from '/Users/geet/Documents/Repositories/Owned/goesdata/notebooks/../src/parallel_plotting.py'>

In [2]:
fs = fsspec.filesystem('s3', anon=True)
urls = []

for i in tqdm(range(210,240)):
    for f in fs.glob(f"s3://noaa-goes16/ABI-L2-CMIPF/2021/{i}/*/*C13*.nc"):
        urls.append('s3://' + f)

100%|██████████| 30/30 [01:54<00:00,  3.83s/it]


In [4]:
len(urls)

4321

In [5]:
client = Client(n_workers=8)
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 8
Total threads: 8,Total memory: 16.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:52607,Workers: 8
Dashboard: http://127.0.0.1:8787/status,Total threads: 8
Started: Just now,Total memory: 16.00 GiB
Comm: tcp://127.0.0.1:52632,Total threads: 1
Dashboard: http://127.0.0.1:52633/status,Memory: 2.00 GiB
Nanny: tcp://127.0.0.1:52610,


In [6]:
def gen_json(u):
    so = dict(
        mode="rb", anon=True, default_fill_cache=False,
        default_cache_type="none"
    )
    with fsspec.open(u, **so) as inf:
        h5chunks = SingleHdf5ToZarr(inf, u, inline_threshold=300)
        if os.path.isfile("jsons/{u.split('/')[-1]}.json"):
            pass
        else:
            with open(f"jsons/{u.split('/')[-1]}.json", 'wb') as outf:
                outf.write(ujson.dumps(h5chunks.translate()).encode())

In [ ]:
pathlib.Path('./jsons/').mkdir(exist_ok=True)

dask.compute(*[dask.delayed(gen_json)(u) for u in urls]);

KeyboardInterrupt: 

2023-06-11 02:17:17,510 - tornado.application - ERROR - Exception in callback <bound method SystemMonitor.update of <SystemMonitor: cpu: 0 memory: 52 MB fds: 257>>
Traceback (most recent call last):
  File "/Users/geet/Documents/Repositories/Owned/goesdata/goesdata_env/lib/python3.10/site-packages/tornado/ioloop.py", line 919, in _run
    val = self.callback()
  File "/Users/geet/Documents/Repositories/Owned/goesdata/goesdata_env/lib/python3.10/site-packages/distributed/system_monitor.py", line 160, in update
    net_ioc = psutil.net_io_counters()
  File "/Users/geet/Documents/Repositories/Owned/goesdata/goesdata_env/lib/python3.10/site-packages/psutil/__init__.py", line 2119, in net_io_counters
    rawdict = _psplatform.net_io_counters()
OSError: [Errno 12] Cannot allocate memory
2023-06-11 03:37:26,249 - tornado.application - ERROR - Exception in callback <bound method SystemMonitor.update of <SystemMonitor: cpu: 0 memory: 34 MB fds: 33>>
Traceback (most recent call last):
  File "/U